In [71]:
import random
import numpy as np

### Задание 1. Шифр Хилла

Возьмем русский алфавит + несколько символов пунктуации и пронумеруем их от 0 до $m - 1$. Чтобы определитель матриц-ключей с большей вероятностью был взаимно простым числом с длиной алфавита $m$, построим наш алфавит так, чтобы его длина была простым числом. В нашем случае число $m = 37$.

In [72]:
alphabet = {
    'а': 0, 'б': 1, 'в': 2, 'г': 3, 'д': 4,
    'е': 5, 'ё': 6, 'ж': 7, 'з': 8, 'и': 9,
    'й': 10, 'к': 11, 'л': 12, 'м': 13, 'н': 14,
    'о': 15, 'п': 16, 'р': 17, 'с': 18, 'т': 19,
    'у': 20, 'ф': 21, 'х': 22, 'ц': 23, 'ч': 24,
    'ш': 25, 'щ': 26, 'ъ': 27, 'ы': 28, 'ь': 29,
    'э': 30, 'ю': 31, 'я': 32, ' ': 33, ',': 34,
    '.': 35, '?': 36
}

inverted_alphabet = {value: key for key, value in alphabet.items()}

alpha_len = len(alphabet)


def letter_to_code(letter):
    return alphabet[letter]


def code_to_letter(code):
    return inverted_alphabet[code]

#### Encrypting
Для шифрования сообщения разобьем его на векторы длины $n$, равной размерности матрицы-ключа (например 2, 3 или 4). Если длина сообщения не делится нацело на такое число, то добавим в конце сообщения пробелы (хоть в случае с 12-символьным сообщением такой проблемы не возникнет).

Каждый вектор умножаем на матрицу-ключ, делим по модулю на длину алфавита и составляем назад из полученных векторов зашифрованное сообщение целиком.

In [73]:
def encrypt_msg(msg, key, n):
    while len(msg) % n != 0:
        msg += ' '

    i = 1
    block = []
    result = ''
    for l in msg:
        code = letter_to_code(l)
        block.append(code)
        if i % n == 0:
            encrypted_vector = key.dot(block) % alpha_len
            for coord in encrypted_vector:
                result += code_to_letter(coord)
            block = []
        i += 1
    return result

Ниже представлен пример 12-символьного сообщения и его шифрования с помощью матриц-ключей (размерностей $n =$2, 3, 4), чьи определители взаимно просты с длиной алфавита.

In [74]:
msg = 'воля и разум'

In [75]:
key2 = np.array([
    [1, 2],
    [3, 4],
])
encrypted_msg2 = encrypt_msg(msg, key2, 2)
encrypted_msg2

'яьвпнчэтпяиб'

In [76]:
key3 = np.array([
    [6, 24, 1],
    [13, 16, 10],
    [20, 17, 15],
])

encrypted_msg3 = encrypt_msg(msg, key3, 3)
encrypted_msg3

'нпюю.дн.чц?ь'

In [77]:
key4 = np.array([
    [11, 2, 3, 4],
    [0, 6, 7, 8],
    [9, 10, 7, 12],
    [13, 3, 1, 16],
])

encrypted_msg4 = encrypt_msg(msg, key4, 4)
encrypted_msg4

'юцжгэнзфр ёэ'

#### Decrypting
Расшифровка производится по алгоритму, аналогичному шифрованию, но с некоторыми важными отличиями.

Если шифрование производится по следующему алгоритму:

$$C = K M \text{ (mod $m$)}$$

где $K$ - матрица-ключ, $M$ - вектор-сообщение, $C$ - зашифрованный вектор, $m$ - длина алфавита (в нашем случае $m = 37$).

То расшифровка выполняется так:

$$M = K^{-1} C \text{ (mod $m$)}$$

где $K^{-1}$ - обратная матрица по модулю $m$ к матрице-ключу.

Нахождение обратной матрицы по модулю похоже на нахождение обычной обратную матрицу. Отличие заключается в том, что полученная присоединенная матрица приводится к остатку от деления на $m$, а вместо деления на определитель, она умножается на обратный элемент к определителю в кольце по модулю $m$.

Для вычисления обратного элемента в кольце по модулю используется расширенный алгоритм Евклида.

In [78]:
def decrypt_msg(msg, key, n):
    if len(msg) % n != 0:
        raise ValueError('incorrect code')
    inverted_key = matrix_mod_inv(key, alpha_len)

    i = 1
    block = []
    result = ''
    for l in msg:
        code = letter_to_code(l)
        block.append(code)
        if i % n == 0:
            original_vector = inverted_key.dot(block) % alpha_len
            for coord in original_vector:
                result += code_to_letter(coord)
            block = []
        i += 1
    return result


def matrix_mod_inv(A, m):
    det = int(np.round(np.linalg.det(A)))
    det_inv = mod_inv(det % m, m)

    adjugate_matrix = np.round(det * np.linalg.inv(A)).astype(int) % m

    inv_matrix = (det_inv * adjugate_matrix) % m
    return inv_matrix


def mod_inv(a, m):
    g, x, y = extended_gcd(a, m)
    if g != 1:
        raise ValueError(f"Обратного элемента для {a} по модулю {m} не существует.")
    else:
        return x % m


def extended_gcd(a, b):
    if a == 0:
        return b, 0, 1
    else:
        g, x, y = extended_gcd(b % a, a)
        return g, y - (b // a) * x, x

Пример расшифровки сообщения для всех ключей

In [79]:
decrypted_msg = decrypt_msg(encrypted_msg2, key2, 2)
print(decrypted_msg)
decrypted_msg = decrypt_msg(encrypted_msg3, key3, 3)
print(decrypted_msg)
decrypted_msg = decrypt_msg(encrypted_msg4, key4, 4)
print(decrypted_msg)

воля и разум
воля и разум
воля и разум


Сымитируем вредоносное вмешательство в зашифрованные сообщения, изменив в каждом сообщении по 3 символа на другие случайные символы из алфавита.

In [80]:
def replace_char_by_index(text, index, replacement):
    return f'{text[:index]}{replacement}{text[index + 1:]}'


def break_message(msg, times=3):
    for i in range(times):
        random_symbol = code_to_letter(random.randint(0, alpha_len - 1))
        random_index = random.randint(0, len(msg) - 1)
        msg = replace_char_by_index(msg, random_index, random_symbol)
    return msg


broken_encrypted_msg2 = break_message(encrypted_msg2)
broken_encrypted_msg3 = break_message(encrypted_msg3)
broken_encrypted_msg4 = break_message(encrypted_msg4)

decrypted_msg = decrypt_msg(broken_encrypted_msg2, key2, 2)
print(decrypted_msg)
decrypted_msg = decrypt_msg(broken_encrypted_msg3, key3, 3)
print(decrypted_msg)
decrypted_msg = decrypt_msg(broken_encrypted_msg4, key4, 4)
print(decrypted_msg)

воц  и раз?е
волтдвбухзум
рю,йтдюдазум


Сообщения с меньшей размерностью ключа меньше подвержены вредоносным вмешательствам, т.к. они делятся на большее количество независимых блоков и фиксированное количество ошибок затрагивает меньший процент от всего сообщения.

### Задание 2. Взлом шифра Хилла

По сути задача взлома шифра сводится к нахождению матрицы-ключа, далее просто применяется механизм расшифровки. Разберемся что для этого нужно и напишем такую функцию.

Просто так из вышеописанного уравнения
$$C = K M \text{ (mod $m$)}$$
при известных векторах $M$ и $C$ невозможно однозначно получить матрицу $K$.

Однако если взять $n$ векторов и представить их в виде матрицы $n \times n$, то для полученной матрицы $M$ получится найти обратную. Тогда нахождение матрицы ключа будет выглядит следующим образом:
$$K = C M^{-1} \text{ (mod $m$)}$$

В связи с этим для взлома ключа появляется ограничение: для составления матрицы необходимо $n$ векторов размерности $n$, т.е. нам нужен оригинал сообщения и его шифр, длина которых должна быть равна как минимум $n^2$ символов.

Исходя из этого, зная оригинал и шифр 12-символьного сообщения, мы можем найти ключ размерности 2 и 3, но не ключ размерности 4, т.к. $12 < 4^2$.

Ещё одно ограничение заключается в том, что матрица $M$ должна быть обратимой в кольце по модулю $m$.

In [81]:
def hack_cipher(encrypted_msg, original_msg, n):
    n_squared = n ** 2
    if len(encrypted_msg) < n_squared or len(original_msg) < n_squared:
        raise ValueError("the key matrix can't be calculated")
    encrypted_msg = encrypted_msg[:n_squared]
    original_msg = original_msg[:n_squared]
    C = np.array(list(map(letter_to_code, encrypted_msg)))
    C.shape = (n, n)
    C = C.transpose()
    P = np.array(list(map(letter_to_code, original_msg)))
    P.shape = (n, n)
    P = P.transpose()
    P_inv = matrix_mod_inv(P, alpha_len)
    return C.dot(P_inv) % alpha_len

Ниже продемонстрировано успешное вычисление ключей-матриц для размерностей 2 и 3.

In [82]:
solved_key2 = hack_cipher(encrypted_msg2, msg, 2)
print(solved_key2)
solved_key3 = hack_cipher(encrypted_msg3, msg, 3)
print(solved_key3)

[[1 2]
 [3 4]]
[[ 6 24  1]
 [13 16 10]
 [20 17 15]]


Однако, как выше упоминалось, вычисление ключа-матрицы размерности 4 невозможно.

In [83]:
solved_key4 = hack_cipher('юцжгэнзфр ёэ', 'воля и разум', 4)

ValueError: the key matrix can't be calculated

### Задание 3. Код Хэмминга

Построим бинарный алфавит для трансляции текста сообщения в поток битов.

In [84]:
binary_alphabet = {
    'А': '00000', 'Б': '00001', 'В': '00010', 'Г': '00011', 'Д': '00100',
    'Е': '00101', 'Ж': '00110', 'З': '00111', 'И': '01000', 'Й': '01001',
    'К': '01010', 'Л': '01011', 'М': '01100', 'Н': '01101', 'О': '01110',
    'П': '01111', 'Р': '10000', 'С': '10001', 'Т': '10010', 'У': '10011',
    'Ф': '10100', 'Х': '10101', 'Ц': '10110', 'Ч': '10111', 'Ш': '11000',
    'Щ': '11001', 'Ъ': '11010', 'Ы': '11011', 'Ь': '11100', 'Э': '11101',
    'Ю': '11110', 'Я': '11111',
}

inverted_binary_alphabet = {value: key for key, value in binary_alphabet.items()}


def translate_text_to_bits(text):
    sequence = ''
    for l in text:
        sequence += binary_alphabet[l]
    return sequence


def translate_bits_to_text(bits):
    i = 1
    letter_code = ''
    text = ''
    for b in bits:
        letter_code += b
        if i % 5 == 0:
            text += inverted_binary_alphabet[letter_code]
            letter_code = ''
        i += 1
    return text

Возьмем сообщение из 4 букв и переведем его в бинарный алфавит.

In [85]:
text_msg = 'МИРА'
msg = translate_text_to_bits(text_msg)
msg

'01100010001000000000'

Кодирование Хэмминга основано на добавлении к сообщению избыточных контрольных битов, которые указывают на четность суммы определенных битов сообщения. Данная операция помогает обнаруживать, а иногда даже исправлять ошибки, возникшие в сообщении при передаче.

В результирующем коде контрольные биты стоят на местах, чьи номера равны степеням двойки. Т.е. контрольный бит $r_i$ стоит на месте $2^{i-1}$. Каждый из них дополняет до четности сумму всех битов из последовательности групп по $i$ бит через каждые $i$ бит, начиная с места этого контрольного бита.

Чтобы контрольный бит дополнял такую сумму, необходимо чтобы он равнялся сумме по модулю 2 всех битов из этой последовательности.

Исходя из этих соображений и строится матрица $G$ для трансформации сообщения в код с контрольными битами. После этого дополнительно в начало кода (т.е. на место с индексом 0) добавляется бит общей четности.

Также присутствует ограничение на длину кодируемой последовательности битов: она должна нацело делиться на кол-во полезных битов в одном блоке (в нашем случае - 4).

In [86]:
def encode_hamming74(msg):
    r = 3
    n = 2 ** r - 1
    k = n - r

    if len(msg) % k != 0:
        raise ValueError('incorrect message length or number of control bits')

    i = 1
    block = []
    result = ''

    G = [
        [1, 1, 1, 0, 0, 0, 0],
        [1, 0, 0, 1, 1, 0, 0],
        [0, 1, 0, 1, 0, 1, 0],
        [1, 1, 0, 1, 0, 0, 1],
    ]

    for b in msg:
        block.append(int(b))
        if i % k == 0:
            encrypted_block = np.dot(block, G) % 2
            total_parity_bit = encrypted_block.sum() % 2
            encrypted_block = np.append(encrypted_block, total_parity_bit)
            encrypted_string = ''.join(encrypted_block.astype(str))
            result += encrypted_string
            block = []
        i += 1
    return result

Закодируем нашу последовательность битов

In [87]:
encoded_msg = encode_hamming74(msg)
encoded_msg

'1100110001010101010101010000000000000000'

Проверка корректности сообщения при декодировании основана на проверки четности определенных последовательностей битов (аналогично кодированию). Четность последовательности вместе с её контрольным битом называется синдромом ошибки $S_i$. Если синдром для каждой последовательности равен нулю, то ошибок в блоке нет.

Также проверяется общая четность блока с помощью соответствующего бита. Это помогает определить общее количество обнаруженных ошибок в блоке. Если оно нечетно, то допуская, что ошибка всего одна, мы можем попробовать её исправить. Если четно, то исправить ошибку невозможно.

В случае одной ошибки, индекс ошибочного бита можно однозначно определить с помощью синдромов ошибки. Он будет равен:
$$j_{err} = \sum_{i=1}^{r} S_i \cdot 2^{i-1}$$

In [104]:
def decode_hamming74(code):
    r = 3
    n = 2 ** r - 1
    k = n - r

    H = [
        [1, 0, 0],
        [0, 1, 0],
        [1, 1, 0],
        [0, 0, 1],
        [1, 0, 1],
        [0, 1, 1],
        [1, 1, 1],
    ]

    i = 1
    block = []
    result = ''
    j = 1
    for b in code:
        block.append(int(b))
        if i % (n + 1) == 0:
            # checking & fixing possible errors
            total_parity = not bool(int(sum(block) % 2))
            del block[-1]  # deleting the total parity bit

            error_syndrome = np.dot(block, H) % 2
            is_error_exist = sum(error_syndrome) > 0
            if is_error_exist:
                if total_parity:
                    print(f'[Блок {j}]: Обнаружено четное количество ошибок: восстановить блок невозможно')
                else:
                    print(f'[Блок {j}]: Обнаружено нечетное количество ошибок: попытка восстановить ошибку')
                    broken_index = int(''.join(map(str, error_syndrome[::-1])), 2) - 1
                    block[broken_index] = int(not bool(block[broken_index]))
            else:
                if total_parity:
                    print(f'[Блок {j}]: Ошибок не обнаружено')
                else:
                    print(f'[Блок {j}]: Ошибка обнаружена в бите общей четности')

            # extracting the content bits
            for power in range(r - 1, -1, -1):
                del block[2 ** power - 1]  # deleting check bits
            result += ''.join(map(str, block))
            block = []
            j += 1
        i += 1
    return result

Удостоверимся, что декодирование работает правильно на невредимом сообщении.

In [106]:
decoded_msg = decode_hamming74(encoded_msg)
translate_bits_to_text(decoded_msg)

[Блок 1]: Ошибок не обнаружено
[Блок 2]: Ошибок не обнаружено
[Блок 3]: Ошибок не обнаружено
[Блок 4]: Ошибок не обнаружено
[Блок 5]: Ошибок не обнаружено


'МИРА'

Сымитируем вредоносное вмешательство, последовательно добавляя от 1 до 4 ошибок в закодированное сообщение.

In [115]:
def beat_the_bit(sequence, index):
    random_bit = bool(int(sequence[index]))
    return replace_char_by_index(sequence, index, str(int(not random_bit)))


def beat_random_bit(sequence):
    random_index = random.randint(0, len(sequence) - 1)
    return beat_the_bit(sequence, random_index)

In [116]:
broken_msg = beat_random_bit(encoded_msg)
translate_bits_to_text(decode_hamming74(broken_msg))

[Блок 1]: Ошибок не обнаружено
[Блок 2]: Ошибок не обнаружено
[Блок 3]: Обнаружено нечетное количество ошибок: попытка восстановить ошибку
[Блок 4]: Ошибок не обнаружено
[Блок 5]: Ошибок не обнаружено


'МИРА'

In [124]:
double_broken_msg = beat_random_bit(beat_random_bit(encoded_msg))
translate_bits_to_text(decode_hamming74(double_broken_msg))

[Блок 1]: Ошибок не обнаружено
[Блок 2]: Обнаружено четное количество ошибок: восстановить блок невозможно
[Блок 3]: Ошибок не обнаружено
[Блок 4]: Ошибок не обнаружено
[Блок 5]: Ошибок не обнаружено


'НАРА'

In [127]:
triple_broken_msg = beat_random_bit(beat_random_bit(beat_random_bit(encoded_msg)))
translate_bits_to_text(decode_hamming74(triple_broken_msg))

[Блок 1]: Обнаружено четное количество ошибок: восстановить блок невозможно
[Блок 2]: Ошибок не обнаружено
[Блок 3]: Ошибок не обнаружено
[Блок 4]: Ошибка обнаружена в бите общей четности
[Блок 5]: Ошибок не обнаружено


'КИРА'

In [129]:
quadruple_broken_msg = beat_random_bit(beat_random_bit(beat_random_bit(beat_random_bit(encoded_msg))))
translate_bits_to_text(decode_hamming74(quadruple_broken_msg))

[Блок 1]: Обнаружено нечетное количество ошибок: попытка восстановить ошибку
[Блок 2]: Ошибок не обнаружено
[Блок 3]: Обнаружено нечетное количество ошибок: попытка восстановить ошибку
[Блок 4]: Обнаружено четное количество ошибок: восстановить блок невозможно
[Блок 5]: Ошибок не обнаружено


'МИТР'

Как и ожидалось, код Хэмминга гарантированно может устранить 1 ошибку в блоке. Если ошибки 3, то код не сможет этого распознать и, допустив, что ошибка всего одна, попытается её исправить. Если же ошибки 2 или 4, то код обнаружит наличие ошибок, но не сможет их исправить.

Интересное замечание состоит в том, что не всегда ошибка в блоке портит само сообщение, т.к. ошибка может оказаться в одном из битов четности. Также важно отметить, что важным при корректировке является именно кол-во ошибок, попавших в один блок, а не во всё сообщение.

Исходя из двух вышеописанных замечаний легко сделать вывод, что устойчивость сообщения закодированного кодом Хэмминга зависит от выбранного размера блоков: если например выбрать тип кода (15, 11) вместо (7, 4), то устойчивость ухудшится. Однако за бо́льшую устойчивость мы платим бо́льшим количеством избыточной информации, а также бо́льшим количеством вычислений при кодировании и декодировании.